
# AdventureWorks KPIs Analysis
Authored by: [Waleed](https://www.linkedin.com/in/waleedabdulla/)

## 1. Objectives
- Provide a __Managerial Overview__ of the KPIs and Business Metrics.

## 2. Data Collection

In [23]:
import pandas as pd
import glob
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

In [30]:
# import the sales dataframes from folder
path = r'../data/Sales'
sales_files = glob.glob(path+'/*.csv')
sales_list = []
for file in sales_files:
    df = pd.read_csv(file, index_col = None, header= 0)
    sales_list.append(df)
sales = pd.concat(sales_list, axis = 0, ignore_index = True)

# import products dataframe from csv file
products = pd.read_csv('../data/AdventureWorks_Products.csv')

#import territories table from csv file
territories = pd.read_csv('../data/AdventureWorks_Territories.csv')

# import categories dataframe from csv file
categories = pd.read_csv('../data/AdventureWorks_Product_Categories.csv')

# import subcategories dataframe from csv file
subcategories = pd.read_csv('../data/AdventureWorks_Product_Subcategories.csv')

# import customers dataframe from csv file
customers = pd.read_csv('../data/AdventureWorks_Customers.csv')


# make a list for tables
dataframes_list = [(sales,'Sales'),(customers,'Customers'),(products,'Products'),(territories,'Territories'),
               (categories,'Categories'),(subcategories,'Subcategories')]

# iterate over the tables-list and print info for each
for dataframe,dataframe_name in dataframes_list:
    print(f"========{dataframe_name.capitalize()}==========")
    print(dataframe.info(),'\n')

========Sales==========
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50466 entries, 0 to 50465
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   OrderDate      50466 non-null  object
 1   StockDate      50466 non-null  object
 2   OrderNumber    50466 non-null  object
 3   ProductKey     50466 non-null  int64 
 4   CustomerKey    50466 non-null  int64 
 5   TerritoryKey   50466 non-null  int64 
 6   OrderLineItem  50466 non-null  int64 
 7   OrderQuantity  50466 non-null  int64 
dtypes: int64(5), object(3)
memory usage: 3.1+ MB
None 

========Customers==========
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18018 entries, 0 to 18017
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   CustomerKey     18018 non-null  int64 
 1   Prefix          18018 non-null  object
 2   FirstName       18018 non-null  object
 3   LastName        180

## 3. Data Preparation

In [33]:


# merge the sales dataframe with the products data frame and assign that to sales_product
data = sales.merge(products,left_on = 'ProductKey', right_on = 'ProductKey')

data = data.merge(customers, left_on = 'CustomerKey', right_on = 'CustomerKey')

data = data.merge(territories, left_on = 'TerritoryKey', right_on = 'SalesTerritoryKey')



data['Revenue'] = round(data['ProductPrice']*data['OrderQuantity'],2)

data['Cost'] = round( data['ProductCost']*data['OrderQuantity'],2)

data['GrossProfit'] = (data['Revenue'] - data['Cost'])

data['GrossMargin'] = round(data['GrossProfit']/data['Revenue'],2)



data['OrderDate'] = pd.to_datetime(data['OrderDate'])

data['StockDate'] = pd.to_datetime(data['StockDate'])

data['ProductShelfTime'] = (data['OrderDate'] - data['StockDate']).astype(str).\
                            str.split(' ').\
                            str.get(0).\
                            astype(int)
req_cols =  ['OrderDate','OrderNumber','ProductKey','Country','Region','OrderLineItem','Cost','OrderQuantity',
             'Revenue','GrossProfit','GrossMargin','ProductShelfTime']
data = data[req_cols]
# data.drop(
#     ['TerritoryKey','ProductSubcategoryKey',
#      'SalesTerritoryKey','ProductSubcategoryKey',
#      'CustomerKey','Country','Continent'],
#     axis=1,inplace=True)

len(data)

50466

In [26]:
# optimazing the storage
categorical_cols_names = [col for col in data.select_dtypes(include= 'O').columns if col != 'OrderNumber']
for col in categorical_cols_names:
    data[col] = data[col].astype('category')

In [27]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50466 entries, 0 to 50465
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   OrderDate         50466 non-null  datetime64[ns]
 1   OrderNumber       50466 non-null  object        
 2   ProductKey        50466 non-null  int64         
 3   Country           50466 non-null  category      
 4   Region            50466 non-null  category      
 5   OrderLineItem     50466 non-null  int64         
 6   Cost              50466 non-null  float64       
 7   OrderQuantity     50466 non-null  int64         
 8   Revenue           50466 non-null  float64       
 9   GrossProfit       50466 non-null  float64       
 10  GrossMargin       50466 non-null  float64       
 11  ProductShelfTime  50466 non-null  int32         
dtypes: category(2), datetime64[ns](1), float64(4), int32(1), int64(3), object(1)
memory usage: 4.1+ MB


## 4. KPIs Analysis

### Totals

__Total Orders__

In [28]:
tot_orders = len(data['OrderNumber'].value_counts().index)
print(f'Total Orders = {tot_orders:,} orders')

Total Orders = 22,693 orders


__Total Quantity Sold__

In [29]:
tot_qty = data['OrderQuantity'].sum()
print(f'Total Quantity Sold = {tot_qty:,} items')

Total Quantity Sold = 75,786 items


__Total Revenue__

In [35]:
tot_rev = data['Revenue'].sum().round(0)
print(f'Total Revenue = ${tot_rev:,}')

Total Revenue = $22,409,162.0


__Total Products Cost__

In [36]:
tot_cost = data['Cost'].sum().round(0)
print(f'Total Cost = ${tot_cost:,}')

Total Cost = $13,000,953.0


__Total Gross Profit__

In [37]:
tot_gross_pro = data['GrossProfit'].sum().round(0)
print(f'Total Gross Profit = ${tot_gross_pro:,}')

Total Gross Profit = $9,408,209.0


### Averages

__Average Order Size__

In [39]:
avg_order_size = round(tot_qty/tot_orders,2)
print(f'Average Order Size = {avg_order_size} items')

Average Order Size = 3.34 items


__Average Revenue per Order__

In [40]:
avg_rev_per_order = round(tot_rev/tot_orders,2)
print(f'Average Revenue per Order = ${avg_rev_per_order}')

Average Revenue per Order = $987.49


__Average Revenue per Item__

In [41]:
avg_rev_per_item = round(tot_rev/tot_qty,2)
print(f'Average Revenue per Itme = ${avg_rev_per_item}')

Average Revenue per Itme = $295.69


__Average Cost per Item__

In [42]:
avg_cost_per_item = round(tot_cost/tot_qty,2)
print(f'Average Cost per Item = ${avg_cost_per_item}')

Average Cost per Item = $171.55


__Average Gross Profit per Order__

In [43]:
avg_gross_pro_per_order = (tot_gross_pro/tot_orders).round(2)
print(f'Average Gross Profit per Order = ${avg_gross_pro_per_order}')

Average Gross Profit per Order = $414.59


__Average Gross Profit per Item__

In [44]:
avg_gross_pro_per_item = (tot_gross_pro/tot_qty).round(2)
print(f'Average Gross Profit per Item = ${avg_gross_pro_per_item}')

Average Gross Profit per Item = $124.14


__Average Self Time__

In [46]:
avg_shelf_time = data['ProductShelfTime'].mean()
print(f'Average Shelf-Time = {avg_shelf_time:.0f} days')

Average Shelf-Time = 4816 days


### Margins

__Gross Margin__

In [47]:
tot_gross_mar = tot_gross_pro/tot_rev
print(f'Total Gross Margin = %{tot_gross_mar*100:.2f}')

Total Gross Margin = %41.98
